In [ ]:
import polars as pl
import numpy as np
import networkx as nx
import random
from math import cos, sin, pi ,sqrt
import rtsvg
rt  = rtsvg.RACETrack()
svg_hdr     = f'<svg x="0" y="0" width="512" height="512" viewbox="0 0 128 128"><rect x="0" y="0" width="128" height="128" fill="white"/>'
svg_ftr     = '</svg>'

In [ ]:
def makeRandomCircles(_num_=4):
    overlap = True
    while overlap:
        circles = []
        for i in range(_num_):
            x = random.uniform(30,98)
            y = random.uniform(30,98)
            r = random.uniform(5,30)
            circles.append((x,y,r))
        overlap = False
        for i in range(_num_):
            for j in range(i+1,_num_):
                if sqrt((circles[i][0]-circles[j][0])**2 + (circles[i][1]-circles[j][1])**2) < (circles[i][2] + circles[j][2] + 10): # make sure they are at least 20 units apart
                    overlap = True
    return circles

tiles = []
for _sample_ in range(90):
    _circles_ = makeRandomCircles(4)
    _svg_     = []
    for _circle_ in _circles_: _svg_.append(f'<circle cx="{_circle_[0]}" cy="{_circle_[1]}" r="{_circle_[2]}" fill="none" stroke="#c0c0c0" stroke-width="1.2"/>')
    _polys_   = rt.isedgarVoronoi(_circles_, [(0,0),(0,128),(128,128),(128,0)], use_circle_radius=True)
    _seen_    = set()
    for _poly_ in _polys_:
        _xy_ = _poly_[0]
        if _xy_ not in _seen_:
            _svg_.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="{1+1.5*random.random()}" fill="none" stroke="black" stroke-width="0.1"/>')
            _seen_.add(_xy_)
        d = f'M {_xy_[0]} {_xy_[1]} '
        for _xy_ in _poly_[1:]:
            if _xy_ not in _seen_:
                _svg_.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="{1+1.5*random.random()}" fill="none" stroke="black" stroke-width="0.1"/>')
                _seen_.add(_xy_) 
            d += f'L {_xy_[0]} {_xy_[1]} '
        d += 'Z'
        _svg_.append(f'<path d="{d}" fill="none" stroke="black" stroke-width="0.5"/>')
    for i in range(len(_circles_)):
        c0, c1, c2 = _circles_[i], _circles_[(i+1)%len(_circles_)], _circles_[(i+2)%len(_circles_)]
        if (c1[1] - c0[1])*(c2[0] - c1[0]) != (c2[1] - c1[1]) * (c1[0] - c0[0]): # not colinear
            _xy_ = rt.approxThreeCirclesCenter(c0, c1, c2)
            _svg_.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="3" fill="none" stroke="red" stroke-width="0.1"/>')
            _svg_.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="0.3" fill="red" stroke="none"/>')
    tiles.append(svg_hdr + ''.join(_svg_) + svg_ftr)
rt.table(tiles, per_row=3,spacer=2)